In [174]:
headers = {
		"Content-Type": "application/json",
		"Connection": "keep-alive",
		"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:120.0) Gecko/20100101 Firefox/120.0",
		"Access-Control-Allow-Origin": "*"
}

import re

def remove_whitespace(s: str, keep_spaces: bool = True) -> str:
    """
    Clean whitespace while keeping natural spaces between words.

    - Convert all non-space whitespace (tabs, newlines, etc.) into a space
    - Collapse multiple spaces into one
    - Trim leading/trailing spaces
    """
    # Replace tabs/newlines/etc. with a space
    s = re.sub(r"[^\S ]+", " ", s)

    # Collapse multiple spaces into one
    s = re.sub(r" +", " ", s)
    
    # Replace \ by ""
    s = re.sub(r"\\", "", s)
    
    # Trim
    return s.strip()


def get_specific_element(regex: str, text: str):
    m = re.search(regex, text)
    return m.group(0) if m else None

def safe_text(element, keep_spaces=False):
    if not element:
        return ""
    return remove_whitespace(element, keep_spaces)




In [175]:
import requests
url = "https://www.paruvendu.fr/immobilier/vente/appartement/1286492157A1KIVHAP000"
response = requests.get(url,headers=headers)
htmlContent = ""

if response.status_code == 200:
    print("Site web chargé")
    htmlContent = response.text
else:
    print("Erreur lors du chargement de la page , Status Code : "+response.status_code)


response


Site web chargé


<Response [200]>

In [180]:
import subprocess
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium import webdriver


opt = Options()
opt.add_argument("start-maximized")
opt.add_argument("--lang=en-US")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=opt)

opt.add_argument("start-maximized")
opt.add_argument("--lang=en-US")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=opt)

driver.get(url)

sleep(2)

driver.find_element(By.XPATH, "//button[text()='Accepter']").click()

try:
    read_more = driver.find_element(By.ID, "linkAnnonceTrunc")
    driver.execute_script("arguments[0].click();", read_more)
    sleep(2)  # On attend man !
except Exception as e:
    print(f"Error: {e}")

element = driver.find_element(By.XPATH, "//div[@class='im12_txt_ann im12_txt_ann_auto']")

description = element.text

# On'se taille
driver.quit()

# on passe a la suite mon gars avec de la bonne soupe !

from bs4 import BeautifulSoup

data = {}
if htmlContent:
    soup = BeautifulSoup(htmlContent, 'html.parser')

    data["title"] = safe_text(soup.find("span", {"id": "detail_h1"}).text, True)

    data["location"] = safe_text(soup.find("span", {"id": "detail_loc"}).text)

    data["descriptionTitle"] = safe_text(
        soup.find("h2", {"class": "autodetail-titre sepdetail14-ssbordure"}).text
    )

    data["description"] = safe_text(
        description
    )
    
    data["nbp"] = get_specific_element(
        r"\d+", 
        safe_text(soup.find("li", {"class": "nbp"}).text)
    )

    data["surf"] = get_specific_element(
        r"\d+",
        safe_text(soup.find("li", {"class": "surf"}).text)
    )

    data["prix"] = get_specific_element(
        r"\d+(?: \d+)*",
        safe_text(soup.find("div", {"class": "prixactionalerte-box"}).text)
    )

    data["enseigneInfosvendeur"] = safe_text(
        soup.find("p", {"class": "enseigne-infosvendeur"}).text
    )

    print( data )


{'title': 'Vente Appartement 4 pièces 90 m²', 'location': 'Santeny (94440)', 'descriptionTitle': 'Description T4 à Santeny', 'description': "Le charme de l'ancien, le confort du moderne SAFTI LIGONNIERE IMMOBILIER vous présente ce charmant triplex entièrement rénové, situé au cœur du village de Santeny. Totalement indépendant, ce bien atypique s'apparente à une maison de ville, offrant le confort d'un appartement sans les charges de copropriété. Idéal pour un jeune couple ou une famille qui recherchent un lieu de vie unique, chaleureux et sans extérieur à entretenir. Cette maison datant de 1890, rénovée avec goût, marie le cachet de l'ancien (poutres, cave voûtée) au confort moderne (design contemporain). Elle se compose au RDC d'une belle pièce de vie lumineuse avec cuisine ouverte aménagée, un cellier avec accès à la cave voûtée permettant du stockage ou l'installation pour les amateurs d'une cave à vin. Au 1er étage deux chambres, dont une avec dressing, une salle d'eau et des toile